## Homework 5 CME306
### Problem 9.11

In [2]:
import numpy as np
import matplotlib.pyplot as plt

#### Definitions

In [5]:
def exact_solution(x, t):
    """
    Given two 1D arrays x and t, computes the value of the exact solution at points x[i], t[i]
    """
    values = np.exp(-np.pi ** 2 * t) * np.sin(np.pi * x) - np.exp(-9 * np.pi ** 2 * t) * np.sin(3 * np.pi * x)
    return values

In [6]:
def v(x):
    """
    Given a 1D array x, computes teh values of v at points x[i]
    """
    values = np.sin(np.pi * x) - np.sin(3 * np.pi * x)
    return values

#### Forward Euler

In [8]:
def forward_euler(k, M, n_max):
    """
    Computes approximations for u(x_j, t_n) using the forward Euler scheme until time step n_max
    """
    # Definitions
    h = 1 / M
    lambd = k / h ** 2
    x = np.arange(1, M) / M
    U = np.zeros((M + 1, n_max + 1)) # U[j, n] will contain an approximation of u(x_j, t_n)
    
    # Initial values
    U[1:-1, 0] = v(x)
    
    # Defining matrix A
    A = np.eye(M - 1) * (1 - 2 * lambd)
    for i in range(M - 2):
        A[i, i + 1], A[i + 1, i] = lambd, lambd
    
    # Iterations
    for i in range(1, n_max + 1):
        U[1:-1, i] = A @ U[1:-1, i - 1]
    
    return U

In [13]:
# h = 1 / 10, k = 1 / 600
M = 10
k = 1 / 600
n_max = 600
U600 = forward_euler(k, M, n_max)
error600 = abs(exact_solution(0.5, 1) - U600[5, 600])
print(f"Error at (1/2, 1) for forward Euler using h = 1 / 10, k = 1 / 600: {error600}")

Error at (1/2, 1) for forward Euler using h = 1 / 10, k = 1 / 600: 9.290894187867103e-09


In [15]:
# h = 1 / 10, k = 1 / 300
M = 10
k = 1 / 300
n_max = 300
U300 = forward_euler(k, M, n_max)
error300 = abs(exact_solution(0.5, 1) - U300[5, 300])
print(f"Error at (1/2, 1) for forward Euler using h = 1 / 10, k = 1 / 300: {error300}")

Error at (1/2, 1) for forward Euler using h = 1 / 10, k = 1 / 300: 4.08879347959175e-06


In [16]:
# h = 1 / 10, k = 1 / 100
M = 10
k = 1 / 100
n_max = 100
U100 = forward_euler(k, M, n_max)
error100 = abs(exact_solution(0.5, 1) - U100[5, 100])
print(f"Error at (1/2, 1) for forward Euler using h = 1 / 10, k = 1 / 100: {error100}")

Error at (1/2, 1) for forward Euler using h = 1 / 10, k = 1 / 100: 5.7256098329672984e+29


We observe that the forward Euler scheme did not converge for $h=\frac{1}{10}$ and $k=\frac{1}{100}$. This is consistent with the material seen in class, where we showed that the method is only stable if $k \leqslant \frac{1}{2}h^2$.

For the two other values of $k$, the scheme converges, and we observe that reducing $k$ by a factor 2 induces a reduction of the error by 40, which is unexpected...

#### Crank Nicolson

In [21]:
def crank_nicolson(k, M, n_max):
    """
    Computes approximations for u(x_j, t_n) using the Crank Nicolson scheme until time step n_max
    """
    # Definitions
    h = 1 / M
    lambd = k / h ** 2
    x = np.arange(1, M) / M
    U = np.zeros((M + 1, n_max + 1)) # U[j, n] will contain an approximation of u(x_j, t_n)
    
    # Initial values
    U[1:-1, 0] = v(x)
    
    # Defining matrix A, B, and inv(B) * A
    A, B = np.eye(M - 1) * (1 - lambd), np.eye(M - 1) * (1 + lambd)
    for i in range(M - 2):
        A[i, i + 1], A[i + 1, i] = 0.5 * lambd, 0.5 * lambd
        B[i, i + 1], B[i + 1, i] = -0.5 * lambd, -0.5 * lambd
        
    C = np.linalg.inv(B) @ A
    
    # Iterations
    for i in range(1, n_max + 1):
        U[1:-1, i] = C @ U[1:-1, i - 1]
    
    return U

In [24]:
# h = 1 / 10, k = 1 / 10
M = 10
k = 1 / 10
n_max = 10
U10 = crank_nicolson(k, M, n_max)
error10 = abs(exact_solution(0.5, 1) - U10[5, 10])
print(f"Error at (1/2, 1) for Crank Nicolson using h = 1 / 10, k = 1 / 10: {error10}")

Error at (1/2, 1) for Crank Nicolson using h = 1 / 10, k = 1 / 10: 0.0070503547775257235


The Crank Nicolson method does not diverge converge even when $k$ has the same order of magnitude as $h$. There is no stiffness! But the error is 100 times greater than the actual value that we want to measure...